In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from joblib import load

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from tensorflow import set_random_seed
set_random_seed(12)

Using TensorFlow backend.
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaco

# Preprocessing

In [2]:
data = pd.read_csv('./datasets/IBM_daily.csv').sort_values(by='date')

In [3]:
data

,date,1. open,2. high,3. low,4. close,5. volume
5217,1999-11-01,98.50,98.8100,96.37,96.75,9551800.0
5216,1999-11-02,96.75,96.8100,93.69,94.81,11105400.0
5215,1999-11-03,95.87,95.9400,93.50,94.37,10369100.0
5214,1999-11-04,94.44,94.4400,90.00,91.56,16697600.0
5213,1999-11-05,92.75,92.9400,90.19,90.25,13737600.0
...,...,...,...,...,...,...
4,2020-07-22,125.90,129.4700,125.80,128.67,8195366.0
3,2020-07-23,129.10,129.3700,127.15,127.33,4220136.0
2,2020-07-24,126.48,127.6459,125.50,125.79,3531076.0
1,2020-07-27,124.86,126.3200,124.71,126.21,3733547.0


In [4]:
data = data.drop(columns='date')
data

,1. open,2. high,3. low,4. close,5. volume
5217,98.50,98.8100,96.37,96.75,9551800.0
5216,96.75,96.8100,93.69,94.81,11105400.0
5215,95.87,95.9400,93.50,94.37,10369100.0
5214,94.44,94.4400,90.00,91.56,16697600.0
5213,92.75,92.9400,90.19,90.25,13737600.0
...,...,...,...,...,...
4,125.90,129.4700,125.80,128.67,8195366.0
3,129.10,129.3700,127.15,127.33,4220136.0
2,126.48,127.6459,125.50,125.79,3531076.0
1,124.86,126.3200,124.71,126.21,3733547.0


In [5]:
# Sequence for the LSTM network
backlook = 92

# Days to predict
days = 2

# Size of data split for testing
train_size = 0.9

In [6]:
# Train-Test split
split = int(len(data) * train_size)

train = data.to_numpy()[:split]
test = data.to_numpy()[split:]

In [7]:
# Load normalisers
normaliser = load('./normalisers/x_normaliser.joblib')
y_normaliser = load('./normalisers/y_normaliser.joblib')

In [8]:
# Normalise data
train_norm = normaliser.transform(train)
test_norm = normaliser.transform(test)

##### Now I get indexes for chunks from 2 in 2 days (history doubles the backlook)

In [9]:
index = []

for i in range(days):
    index.append(np.arange(i,len(train),days))

In [10]:
ordered_index = []

for i in range(len(index[0]) - backlook):
    for e in range(days):
        try: ordered_index.append(index[e][i:i+backlook])
        except: pass

In [11]:
# Normalised train chunks
X_train = np.array([train_norm[ix].copy() for ix in ordered_index])
Y_train = np.array([train_norm[ordered_index[i+days][-1],3].copy() for i in range(len(ordered_index) - days)])
Y_train = np.expand_dims(Y_train, -1)

In [12]:
# Delete those sequences that doesn't have a -th day in the results
X_train = X_train[:Y_train.shape[0]]

##### Same for test

In [13]:
index = []

for i in range(days):
    index.append(np.arange(i,len(test),days))

In [14]:
ordered_index = []

for i in range(len(index[0]) - backlook):
    for e in range(days):
        try: ordered_index.append(index[e][i:i+backlook])
        except: pass

In [15]:
# Normalised test chunks
X_test = np.array([test_norm[ix].copy() for ix in ordered_index])
Y_test = np.array([test_norm[ordered_index[i+days][-1],3].copy() for i in range(len(ordered_index) - days)])
Y_test = np.expand_dims(Y_test, -1)

In [16]:
# Delete those sequences that doesn't have a -th day in the results
X_test = X_test[:Y_test.shape[0]]

# Model selection

##### Model constructor

In [17]:
def build_lstm(x, y, batch_size=512, epochs=24, **params):
    
    # List of parameters
    if 'lstmsize' not in params: params['lstmsize'] = x.shape[1]
    if 'density' not in params: params['density'] = int((params['lstmsize']//1.5)*2)
    if 'activation' not in params: params['activation'] = 'relu'
    if 'twice' not in params: params['twice'] = False
    if 'optimizer' not in params: params['optimizer'] = 'adam'
    if 'shuffle' not in params: params['shuffle'] = False
    
    # Model definition
    model = Sequential()
    
    model.add(LSTM(params['lstmsize'], input_shape=x.shape[1:], return_sequences=params['twice']))
    
    if 'dropout' in params:
        model.add(Dropout(params['dropout']))
    
    if params['twice']:
        model.add(LSTM(params['lstmsize']))
        
        if 'dropout' in params:
            model.add(Dropout(params['dropout']))
            
    model.add(Dense(params['density'], activation=params['activation']))
    
    if 'full_density' in params and params['full_density']:
        density = params['density']//2
        while density >= 12:
            model.add(Dense(density, activation=params['activation']))
            density //= 2
            
    model.add(Dense(1, activation='linear'))
    
    model.compile(loss='mse', optimizer=params['optimizer'])
    
    if 'callbacks' in params:
        callback = model.fit(x=x, y=y, validation_split=0.1, batch_size=batch_size, epochs=epochs, shuffle=params['shuffle'],
                            callbacks=params['callbacks'])
    else:
        callback = model.fit(x=x, y=y, validation_split=0.1, batch_size=batch_size, epochs=epochs, shuffle=params['shuffle'])
    
    return [model, callback, params]

##### Evolutive algorith to search for the most optimal model

In [18]:
def set_genes(x, y, population_size, population=[]):
    if population_size-len(population) < 0:
        print('population size must be greater or equal than actual population')
        return
    
    if len(population) > 0:
        if len(population) < 3:
            print('population should be empty or 3 at least')
            return
        
        population[2] = breed_genes(x, y, population[0][2].copy())
        population[1] = breed_genes(x, y, combine=(population[0][2], population[1][2]))
        population = population[:3]
    
    for _ in range(population_size-len(population)):
        subject = breed_genes(x, y)
        population.append(subject)
        
    return population

def breed_genes(x, y, genes={}, combine=None):
    genes['x'] = x
    genes['y'] = y
    
    if type(combine) is list or type(combine) is tuple:
        genes['lstmsize'] = combine[np.random.randint(0,2)]['lstmsize']
        genes['density'] = combine[np.random.randint(0,2)]['density']
        genes['activation'] = combine[np.random.randint(0,2)]['activation']
        genes['twice'] = combine[np.random.randint(0,2)]['twice']
        genes['optimizer'] = combine[np.random.randint(0,2)]['optimizer']
        genes['shuffle'] = combine[np.random.randint(0,2)]['shuffle']
        
        genes['dropout'] = combine[np.random.randint(0,2)].get('dropout')
        if genes['dropout'] is None: del genes['dropout']
        genes['full_density'] = combine[np.random.randint(0,2)].get('full_density')
        if genes['full_density'] is None: del genes['full_density']
            
    else:
        if np.random.randint(0,3) == 1:
            genes['lstmsize'] = int((np.random.randint(x.shape[1],x.shape[1]*2)//2)*2)

        if np.random.randint(0,3) == 1:
            genes['density'] = int((np.random.randint(x.shape[1]//2,x.shape[1]*2.66)//2)*2)

        if np.random.randint(0,3) == 1:
            activation = [
                'relu',
                'sigmoid',
                'softplus',
                'softsign',
                'tanh',
                'selu',
                'elu',
                'exponential'
            ]
            genes['activation'] = activation[np.random.randint(0,8)]

        if np.random.randint(0,3) == 1:
            genes['twice'] = True

        if np.random.randint(0,3) == 1:
            optimizer = [
                'sgd',
                'rmsprop',
                'adam',
                'adadelta',
                'adagrad',
                'adamax',
                'nadam'
            ]
            genes['optimizer'] = optimizer[np.random.randint(0,7)]

        if np.random.randint(0,3) == 1:
            genes['shuffle'] = True

        if np.random.randint(0,3) == 1:
            genes['dropout'] = np.random.randint(1,4)/10

        if np.random.randint(0,3) == 1:
            genes['full_density'] = True
            
    new_model = build_lstm(**genes)
        
    return new_model

In [19]:
def breed_population(x, y, generations, population_size, population=[]):
    
    if generations > 1 and population_size < 3:
        print('population size should be of a minimum of 3 for more than one generation')
        return
    
    for g in range(generations):
        print(f'\nGENERATION {g}\n')
        population = set_genes(x, y, population_size, population)
        population = sorted(population, key=lambda x: x[1].history['val_loss'][-1]+x[1].history['loss'][-1])
        
    return population

In [20]:
best_models = breed_population(X_train, Y_train, generations=20, population_size=10)


GENERATION 0

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 4059 samples, validate on 451 samples
Epoch 1/24
4059/4059 [==============================] - 2s 557us/step - loss: 0.2160 - val_loss: 0.1692
Epoch 2/24
4059/4059 [==============================] - 1s 315us/step - loss: 0.0960 - val_loss: 0.0663
Epoch 3/24
4059/4059 [==============================] - 1s 312us/step - loss: 0.0546 - val_loss: 0.0267
Epoch 4/24
4059/4059 [==============================] - 1s 320us/step - loss: 0.0395 - val_loss: 0.0121
Epoch 5/24
4059/4059 [==============================] - 1s 312us/step - loss: 0.0352 - val_loss: 0.0066
Epoch 6/24
4059/4059 [==============================] - 1s 323us/step - loss: 0.0323 - val_loss: 0.0044
Epoch 7/24
4059/4059 [==============================] - 1s 322us/step - loss: 0.0296 - val_loss: 0.0034
Epoch 8/24
4059/4059 [==============================] - 1s 320us/step - loss: 0.0274 - val

4059/4059 [==============================] - 1s 317us/step - loss: 0.0032 - val_loss: 0.0019
Train on 4059 samples, validate on 451 samples
Epoch 1/24
4059/4059 [==============================] - 3s 736us/step - loss: 0.8218 - val_loss: 0.0090
Epoch 2/24
4059/4059 [==============================] - 2s 533us/step - loss: 0.0165 - val_loss: 0.0024
Epoch 3/24
4059/4059 [==============================] - 2s 536us/step - loss: 0.0082 - val_loss: 0.0015
Epoch 4/24
4059/4059 [==============================] - 2s 532us/step - loss: 0.0059 - val_loss: 0.0019
Epoch 5/24
4059/4059 [==============================] - 2s 531us/step - loss: 0.0051 - val_loss: 0.0019
Epoch 6/24
4059/4059 [==============================] - 2s 535us/step - loss: 0.0050 - val_loss: 0.0019
Epoch 7/24
4059/4059 [==============================] - 2s 528us/step - loss: 0.0046 - val_loss: 0.0017
Epoch 8/24
4059/4059 [==============================] - 2s 533us/step - loss: 0.0045 - val_loss: 0.0018
Epoch 9/24
4059/4059 [======

4059/4059 [==============================] - 1s 359us/step - loss: 0.0558 - val_loss: 0.0397
Epoch 5/24
4059/4059 [==============================] - 1s 353us/step - loss: 0.0213 - val_loss: 0.0208
Epoch 6/24
4059/4059 [==============================] - 1s 358us/step - loss: 0.0255 - val_loss: 0.0422
Epoch 7/24
4059/4059 [==============================] - 1s 353us/step - loss: 0.0362 - val_loss: 0.0519
Epoch 8/24
4059/4059 [==============================] - 1s 354us/step - loss: 0.0332 - val_loss: 0.0492
Epoch 9/24
4059/4059 [==============================] - 1s 355us/step - loss: 0.0381 - val_loss: 0.0573
Epoch 10/24
4059/4059 [==============================] - 1s 358us/step - loss: 0.0473 - val_loss: 0.0648
Epoch 11/24
4059/4059 [==============================] - 1s 361us/step - loss: 0.0444 - val_loss: 0.0554
Epoch 12/24
4059/4059 [==============================] - 1s 357us/step - loss: 0.0454 - val_loss: 0.0663
Epoch 13/24
4059/4059 [==============================] - 1s 352us/step -

4059/4059 [==============================] - 1s 354us/step - loss: 0.0032 - val_loss: 0.0014
Epoch 10/24
4059/4059 [==============================] - 1s 356us/step - loss: 0.0022 - val_loss: 0.0015
Epoch 11/24
4059/4059 [==============================] - 1s 351us/step - loss: 0.0020 - val_loss: 0.0024
Epoch 12/24
4059/4059 [==============================] - 1s 353us/step - loss: 0.0020 - val_loss: 0.0025
Epoch 13/24
4059/4059 [==============================] - 1s 362us/step - loss: 0.0020 - val_loss: 0.0015
Epoch 14/24
4059/4059 [==============================] - 1s 350us/step - loss: 0.0020 - val_loss: 0.0022
Epoch 15/24
4059/4059 [==============================] - 1s 357us/step - loss: 0.0020 - val_loss: 0.0021
Epoch 16/24
4059/4059 [==============================] - 1s 352us/step - loss: 0.0019 - val_loss: 0.0017
Epoch 17/24
4059/4059 [==============================] - 1s 353us/step - loss: 0.0020 - val_loss: 0.0015
Epoch 18/24
4059/4059 [==============================] - 1s 354us/s

Epoch 14/24
4059/4059 [==============================] - 2s 556us/step - loss: 0.0026 - val_loss: 0.0017
Epoch 15/24
4059/4059 [==============================] - 2s 554us/step - loss: 0.0025 - val_loss: 0.0018
Epoch 16/24
4059/4059 [==============================] - 2s 552us/step - loss: 0.0027 - val_loss: 0.0017
Epoch 17/24
4059/4059 [==============================] - 2s 554us/step - loss: 0.0025 - val_loss: 0.0018
Epoch 18/24
4059/4059 [==============================] - 2s 553us/step - loss: 0.0026 - val_loss: 0.0018
Epoch 19/24
4059/4059 [==============================] - 2s 558us/step - loss: 0.0026 - val_loss: 0.0019
Epoch 20/24
4059/4059 [==============================] - 2s 554us/step - loss: 0.0025 - val_loss: 0.0017
Epoch 21/24
4059/4059 [==============================] - 2s 553us/step - loss: 0.0024 - val_loss: 0.0018
Epoch 22/24
4059/4059 [==============================] - 2s 549us/step - loss: 0.0025 - val_loss: 0.0018
Epoch 23/24
4059/4059 [==============================] 

...

Epoch 16/24
4059/4059 [==============================] - 2s 592us/step - loss: 0.0015 - val_loss: 0.0012
Epoch 17/24
4059/4059 [==============================] - 2s 587us/step - loss: 0.0015 - val_loss: 0.0012
Epoch 18/24
4059/4059 [==============================] - 2s 583us/step - loss: 0.0015 - val_loss: 0.0011
Epoch 19/24
4059/4059 [==============================] - 2s 584us/step - loss: 0.0014 - val_loss: 0.0011
Epoch 20/24
4059/4059 [==============================] - 2s 585us/step - loss: 0.0014 - val_loss: 0.0011
Epoch 21/24
4059/4059 [==============================] - 2s 594us/step - loss: 0.0014 - val_loss: 0.0011
Epoch 22/24
4059/4059 [==============================] - 2s 582us/step - loss: 0.0014 - val_loss: 9.4689e-04
Epoch 23/24
4059/4059 [==============================] - 2s 586us/step - loss: 0.0013 - val_loss: 0.0010
Epoch 24/24
4059/4059 [==============================] - 2s 584us/step - loss: 0.0014 - val_loss: 9.7038e-04
Train on 4059 samples, validate on 451 samples


4059/4059 [==============================] - 2s 415us/step - loss: 0.0018 - val_loss: 0.0014
Epoch 21/24
4059/4059 [==============================] - 2s 419us/step - loss: 0.0018 - val_loss: 0.0013
Epoch 22/24
4059/4059 [==============================] - 2s 412us/step - loss: 0.0018 - val_loss: 0.0013
Epoch 23/24
4059/4059 [==============================] - 2s 414us/step - loss: 0.0018 - val_loss: 0.0013
Epoch 24/24
4059/4059 [==============================] - 2s 412us/step - loss: 0.0017 - val_loss: 0.0013
Train on 4059 samples, validate on 451 samples
Epoch 1/24
4059/4059 [==============================] - 50s 12ms/step - loss: 0.0727 - val_loss: 0.0011
Epoch 2/24
4059/4059 [==============================] - 2s 411us/step - loss: 0.0192 - val_loss: 0.0017
Epoch 3/24
4059/4059 [==============================] - 2s 420us/step - loss: 0.0027 - val_loss: 0.0016
Epoch 4/24
4059/4059 [==============================] - 2s 416us/step - loss: 0.0021 - val_loss: 0.0064
Epoch 5/24
4059/4059 [==


GENERATION 19

Train on 4059 samples, validate on 451 samples
Epoch 1/24
4059/4059 [==============================] - 52s 13ms/step - loss: 0.0788 - val_loss: 0.0047
Epoch 2/24
4059/4059 [==============================] - 2s 590us/step - loss: 0.0114 - val_loss: 0.0045
Epoch 3/24
4059/4059 [==============================] - 2s 589us/step - loss: 0.0057 - val_loss: 0.0014
Epoch 4/24
4059/4059 [==============================] - 2s 590us/step - loss: 0.0040 - val_loss: 0.0057
Epoch 5/24
4059/4059 [==============================] - 2s 586us/step - loss: 0.0029 - val_loss: 0.0010
Epoch 6/24
4059/4059 [==============================] - 2s 583us/step - loss: 0.0023 - val_loss: 9.5067e-04
Epoch 7/24
4059/4059 [==============================] - 2s 586us/step - loss: 0.0021 - val_loss: 0.0017
Epoch 8/24
4059/4059 [==============================] - 2s 585us/step - loss: 0.0020 - val_loss: 0.0013
Epoch 9/24
4059/4059 [==============================] - 2s 591us/step - loss: 0.0017 - val_loss: 0.00

Epoch 4/24
4059/4059 [==============================] - 2s 582us/step - loss: 0.0028 - val_loss: 0.0053
Epoch 5/24
4059/4059 [==============================] - 2s 582us/step - loss: 0.0022 - val_loss: 0.0014
Epoch 6/24
4059/4059 [==============================] - 2s 584us/step - loss: 0.0018 - val_loss: 8.7011e-04
Epoch 7/24
4059/4059 [==============================] - 2s 583us/step - loss: 0.0016 - val_loss: 0.0010
Epoch 8/24
4059/4059 [==============================] - 2s 582us/step - loss: 0.0015 - val_loss: 0.0012
Epoch 9/24
4059/4059 [==============================] - 2s 581us/step - loss: 0.0014 - val_loss: 0.0011
Epoch 10/24
4059/4059 [==============================] - 2s 582us/step - loss: 0.0013 - val_loss: 8.7304e-04
Epoch 11/24
4059/4059 [==============================] - 2s 582us/step - loss: 0.0013 - val_loss: 8.1520e-04
Epoch 12/24
4059/4059 [==============================] - 2s 581us/step - loss: 0.0012 - val_loss: 8.1362e-04
Epoch 13/24
4059/4059 [======================

4059/4059 [==============================] - 2s 587us/step - loss: 0.0014 - val_loss: 0.0011
Epoch 8/24
4059/4059 [==============================] - 2s 588us/step - loss: 0.0013 - val_loss: 0.0013
Epoch 9/24
4059/4059 [==============================] - 2s 590us/step - loss: 0.0013 - val_loss: 0.0017
Epoch 10/24
4059/4059 [==============================] - 2s 600us/step - loss: 0.0012 - val_loss: 0.0014
Epoch 11/24
4059/4059 [==============================] - 2s 593us/step - loss: 0.0012 - val_loss: 0.0013
Epoch 12/24
4059/4059 [==============================] - 2s 587us/step - loss: 0.0012 - val_loss: 9.5796e-04
Epoch 13/24
4059/4059 [==============================] - 2s 590us/step - loss: 0.0011 - val_loss: 8.8506e-04
Epoch 14/24
4059/4059 [==============================] - 2s 585us/step - loss: 0.0011 - val_loss: 0.0011
Epoch 15/24
4059/4059 [==============================] - 2s 586us/step - loss: 0.0011 - val_loss: 0.0011
Epoch 16/24
4059/4059 [==============================] - 2s 5

In [21]:
best_models[0][1].history['val_loss']

[0.013227377086877823,
 0.007363521493971348,
 0.006701738107949495,
 0.003694563638418913,
 0.002590484917163849,
 0.0015512638492509723,
 0.0012072541285306215,
 0.0012590375263243914,
 0.0011046809377148747,
 0.0008431740570813417,
 0.0007672550855204463,
 0.0009459569118916988,
 0.0008021177491173148,
 0.0007410324178636074,
 0.0011878845980390906,
 0.0007233860669657588,
 0.0007533943862654269,
 0.0007234434597194195,
 0.0008392457384616137,
 0.0007401793263852596,
 0.0006948179798200727,
 0.0006939750164747238,
 0.00076015864033252,
 0.0006735030910931528]

In [22]:
print('optimal parameters')
print('_______________________')
for key,val in best_models[0][2].items():
    if key != 'x' and key != 'y': print(f'{key}: {val}')

optimal parameters
_______________________
density: 238
twice: True
optimizer: adam
dropout: 0.1
activation: elu
full_density: True
shuffle: True
lstmsize: 176


In [23]:
best_models[0][0].summary()

Model: "sequential_147"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_293 (LSTM)              (None, 92, 176)           128128    
_________________________________________________________________
dropout_293 (Dropout)        (None, 92, 176)           0         
_________________________________________________________________
lstm_294 (LSTM)              (None, 176)               248512    
_________________________________________________________________
dropout_294 (Dropout)        (None, 176)               0         
_________________________________________________________________
dense_755 (Dense)            (None, 238)               42126     
_________________________________________________________________
dense_756 (Dense)            (None, 119)               28441     
_________________________________________________________________
dense_757 (Dense)            (None, 59)             

In [28]:
ibm_checkpoint = ModelCheckpoint(
    filepath='./checkpoints/IBM.{epoch}-{val_loss}.hdf5',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)

In [29]:
params = best_models[0][2]
params['x'] = X_train
params['y'] = Y_train
params['epochs'] = 2000
params['callbacks'] = [ibm_checkpoint]

In [ ]:
final_model = build_lstm(**params)

Train on 4059 samples, validate on 451 samples
Epoch 1/2000
4059/4059 [==============================] - 58s 14ms/step - loss: 0.0518 - val_loss: 0.0168
Epoch 2/2000
4059/4059 [==============================] - 2s 600us/step - loss: 0.0074 - val_loss: 0.0145
Epoch 3/2000
4059/4059 [==============================] - 2s 591us/step - loss: 0.0040 - val_loss: 0.0064
Epoch 4/2000
4059/4059 [==============================] - 2s 593us/step - loss: 0.0028 - val_loss: 0.0012
Epoch 5/2000
4059/4059 [==============================] - 2s 596us/step - loss: 0.0021 - val_loss: 8.4252e-04
Epoch 6/2000
4059/4059 [==============================] - 2s 593us/step - loss: 0.0017 - val_loss: 8.5756e-04
Epoch 7/2000
4059/4059 [==============================] - 2s 593us/step - loss: 0.0016 - val_loss: 9.0598e-04
Epoch 8/2000
4059/4059 [==============================] - 2s 595us/step - loss: 0.0015 - val_loss: 9.3380e-04
Epoch 9/2000
4059/4059 [==============================] - 2s 596us/step - loss: 0.0014 - 

Epoch 73/2000
4059/4059 [==============================] - 2s 592us/step - loss: 6.5430e-04 - val_loss: 4.8902e-04
Epoch 74/2000
4059/4059 [==============================] - 2s 598us/step - loss: 6.4437e-04 - val_loss: 4.9532e-04
Epoch 75/2000
4059/4059 [==============================] - 2s 590us/step - loss: 6.3573e-04 - val_loss: 5.1657e-04
Epoch 76/2000
4059/4059 [==============================] - 2s 591us/step - loss: 6.3423e-04 - val_loss: 6.8315e-04
Epoch 77/2000
4059/4059 [==============================] - 2s 600us/step - loss: 6.4113e-04 - val_loss: 4.6870e-04
Epoch 78/2000
4059/4059 [==============================] - 2s 602us/step - loss: 6.4582e-04 - val_loss: 7.0944e-04
Epoch 79/2000
4059/4059 [==============================] - 2s 601us/step - loss: 6.7559e-04 - val_loss: 4.7165e-04
Epoch 80/2000
4059/4059 [==============================] - 2s 595us/step - loss: 6.4249e-04 - val_loss: 4.8498e-04
Epoch 81/2000
4059/4059 [==============================] - 2s 595us/step - loss:

Epoch 144/2000
4059/4059 [==============================] - 3s 618us/step - loss: 4.9116e-04 - val_loss: 3.8909e-04
Epoch 145/2000
4059/4059 [==============================] - 2s 614us/step - loss: 5.2973e-04 - val_loss: 5.0895e-04
Epoch 146/2000
4059/4059 [==============================] - 2s 604us/step - loss: 5.7741e-04 - val_loss: 5.1470e-04
Epoch 147/2000
4059/4059 [==============================] - 2s 606us/step - loss: 5.5633e-04 - val_loss: 3.8677e-04
Epoch 148/2000
4059/4059 [==============================] - 2s 605us/step - loss: 5.4546e-04 - val_loss: 4.7200e-04
Epoch 149/2000
4059/4059 [==============================] - 2s 598us/step - loss: 5.3953e-04 - val_loss: 5.2811e-04
Epoch 150/2000
4059/4059 [==============================] - 2s 598us/step - loss: 5.0011e-04 - val_loss: 4.7058e-04
Epoch 151/2000
4059/4059 [==============================] - 2s 596us/step - loss: 4.8625e-04 - val_loss: 3.5230e-04
Epoch 152/2000
4059/4059 [==============================] - 2s 596us/ste

4059/4059 [==============================] - 2s 596us/step - loss: 4.0387e-04 - val_loss: 2.8229e-04
Epoch 215/2000
4059/4059 [==============================] - 2s 596us/step - loss: 3.9265e-04 - val_loss: 4.7618e-04
Epoch 216/2000
4059/4059 [==============================] - 2s 593us/step - loss: 4.3344e-04 - val_loss: 3.4270e-04
Epoch 217/2000
4059/4059 [==============================] - 2s 595us/step - loss: 4.6121e-04 - val_loss: 3.4668e-04
Epoch 218/2000
4059/4059 [==============================] - 2s 597us/step - loss: 4.2145e-04 - val_loss: 3.2769e-04
Epoch 219/2000
4059/4059 [==============================] - 2s 597us/step - loss: 4.3641e-04 - val_loss: 2.7184e-04
Epoch 220/2000
4059/4059 [==============================] - 2s 594us/step - loss: 4.2092e-04 - val_loss: 2.8493e-04
Epoch 221/2000
4059/4059 [==============================] - 2s 596us/step - loss: 4.8161e-04 - val_loss: 3.7004e-04
Epoch 222/2000
4059/4059 [==============================] - 2s 596us/step - loss: 4.438

Epoch 285/2000
4059/4059 [==============================] - 2s 596us/step - loss: 3.9732e-04 - val_loss: 2.7877e-04
Epoch 286/2000
4059/4059 [==============================] - 2s 599us/step - loss: 3.8774e-04 - val_loss: 2.8316e-04
Epoch 287/2000
4059/4059 [==============================] - 2s 598us/step - loss: 3.8481e-04 - val_loss: 3.0711e-04
Epoch 288/2000
4059/4059 [==============================] - 2s 596us/step - loss: 3.6389e-04 - val_loss: 2.5927e-04
Epoch 289/2000
4059/4059 [==============================] - 2s 598us/step - loss: 3.5773e-04 - val_loss: 2.5904e-04
Epoch 290/2000
4059/4059 [==============================] - 2s 598us/step - loss: 3.6566e-04 - val_loss: 3.0336e-04
Epoch 291/2000
4059/4059 [==============================] - 2s 600us/step - loss: 3.6942e-04 - val_loss: 3.3045e-04
Epoch 292/2000
4059/4059 [==============================] - 2s 599us/step - loss: 3.9916e-04 - val_loss: 2.6113e-04
Epoch 293/2000
4059/4059 [==============================] - 2s 595us/ste

...

4059/4059 [==============================] - 2s 596us/step - loss: 3.5111e-04 - val_loss: 3.7526e-04
Epoch 1131/2000
4059/4059 [==============================] - 2s 593us/step - loss: 3.5217e-04 - val_loss: 2.7509e-04
Epoch 1132/2000
4059/4059 [==============================] - 2s 592us/step - loss: 3.3528e-04 - val_loss: 2.7160e-04
Epoch 1133/2000
4059/4059 [==============================] - 2s 595us/step - loss: 3.2068e-04 - val_loss: 2.7100e-04
Epoch 1134/2000
4059/4059 [==============================] - 2s 591us/step - loss: 3.1750e-04 - val_loss: 2.6733e-04
Epoch 1135/2000
4059/4059 [==============================] - 2s 590us/step - loss: 3.0996e-04 - val_loss: 2.6850e-04
Epoch 1136/2000
4059/4059 [==============================] - 2s 590us/step - loss: 3.2798e-04 - val_loss: 2.6961e-04
Epoch 1137/2000
4059/4059 [==============================] - 2s 591us/step - loss: 3.2401e-04 - val_loss: 2.8587e-04
Epoch 1138/2000
4059/4059 [==============================] - 2s 592us/step - los

4059/4059 [==============================] - 2s 599us/step - loss: 3.0127e-04 - val_loss: 2.7197e-04
Epoch 1201/2000
4059/4059 [==============================] - 2s 592us/step - loss: 3.0342e-04 - val_loss: 3.2657e-04
Epoch 1202/2000
4059/4059 [==============================] - 2s 595us/step - loss: 3.1618e-04 - val_loss: 3.5935e-04
Epoch 1203/2000
4059/4059 [==============================] - 2s 599us/step - loss: 3.4434e-04 - val_loss: 3.1067e-04
Epoch 1204/2000
4059/4059 [==============================] - 2s 589us/step - loss: 3.2181e-04 - val_loss: 2.7458e-04
Epoch 1205/2000
4059/4059 [==============================] - 2s 590us/step - loss: 2.9426e-04 - val_loss: 2.6527e-04
Epoch 1206/2000
4059/4059 [==============================] - 2s 595us/step - loss: 2.9293e-04 - val_loss: 2.6631e-04
Epoch 1207/2000
4059/4059 [==============================] - 2s 594us/step - loss: 3.0752e-04 - val_loss: 2.7146e-04
Epoch 1208/2000
4059/4059 [==============================] - 2s 590us/step - los

4059/4059 [==============================] - 2s 593us/step - loss: 3.0317e-04 - val_loss: 2.9547e-04
Epoch 1271/2000
4059/4059 [==============================] - 2s 597us/step - loss: 3.2440e-04 - val_loss: 2.6461e-04
Epoch 1272/2000
4059/4059 [==============================] - 2s 594us/step - loss: 3.0068e-04 - val_loss: 2.7684e-04
Epoch 1273/2000
4059/4059 [==============================] - 2s 597us/step - loss: 2.9664e-04 - val_loss: 4.0242e-04
Epoch 1274/2000
4059/4059 [==============================] - 2s 598us/step - loss: 3.3581e-04 - val_loss: 3.9888e-04
Epoch 1275/2000
4059/4059 [==============================] - 2s 593us/step - loss: 3.1269e-04 - val_loss: 3.0898e-04
Epoch 1276/2000
4059/4059 [==============================] - 2s 595us/step - loss: 3.2273e-04 - val_loss: 2.6029e-04
Epoch 1277/2000
4059/4059 [==============================] - 2s 594us/step - loss: 3.2541e-04 - val_loss: 3.6472e-04
Epoch 1278/2000
4059/4059 [==============================] - 2s 601us/step - los

4059/4059 [==============================] - 2s 591us/step - loss: 2.9976e-04 - val_loss: 3.3379e-04
Epoch 1341/2000
4059/4059 [==============================] - 2s 597us/step - loss: 3.1212e-04 - val_loss: 3.5383e-04
Epoch 1342/2000
4059/4059 [==============================] - 2s 594us/step - loss: 3.1483e-04 - val_loss: 3.3544e-04
Epoch 1343/2000
4059/4059 [==============================] - 2s 599us/step - loss: 2.9648e-04 - val_loss: 2.9528e-04
Epoch 1344/2000
4059/4059 [==============================] - 2s 593us/step - loss: 2.9508e-04 - val_loss: 2.9524e-04
Epoch 1345/2000
4059/4059 [==============================] - 2s 594us/step - loss: 3.2808e-04 - val_loss: 2.6713e-04
Epoch 1346/2000
4059/4059 [==============================] - 2s 595us/step - loss: 3.0317e-04 - val_loss: 3.1283e-04
Epoch 1347/2000
4059/4059 [==============================] - 2s 598us/step - loss: 3.1198e-04 - val_loss: 2.9765e-04
Epoch 1348/2000
4059/4059 [==============================] - 2s 592us/step - los

4059/4059 [==============================] - 2s 592us/step - loss: 3.1543e-04 - val_loss: 3.0571e-04
Epoch 1411/2000
4059/4059 [==============================] - 2s 589us/step - loss: 2.9991e-04 - val_loss: 2.8380e-04
Epoch 1412/2000
4059/4059 [==============================] - 2s 590us/step - loss: 3.3672e-04 - val_loss: 4.4870e-04
Epoch 1413/2000
4059/4059 [==============================] - 2s 591us/step - loss: 3.4496e-04 - val_loss: 2.8873e-04
Epoch 1414/2000
4059/4059 [==============================] - 2s 591us/step - loss: 3.0093e-04 - val_loss: 2.9075e-04
Epoch 1415/2000
4059/4059 [==============================] - 2s 590us/step - loss: 2.8282e-04 - val_loss: 3.2961e-04
Epoch 1416/2000
4059/4059 [==============================] - 2s 589us/step - loss: 2.8841e-04 - val_loss: 2.7456e-04
Epoch 1417/2000
4059/4059 [==============================] - 2s 591us/step - loss: 2.8037e-04 - val_loss: 2.9113e-04
Epoch 1418/2000
4059/4059 [==============================] - 2s 593us/step - los

In [ ]:
final_model[2]

In [ ]:
final_model[0].summary()

In [ ]:
final_model[0].load_weights('./checkpoints/IBM_2days/IBM.(best).hdf5')
y_predicted = final_model[0].predict(X_test)

In [ ]:
predicted_y_test = y_normaliser.inverse_transform(y_predicted)
true_y_test = y_normaliser.inverse_transform(Y_test)

In [ ]:
loss = np.mean((true_y_test - predicted_y_test)**2)
medium_error = np.mean(abs(true_y_test - predicted_y_test))
print(f'MSE is {loss:.2f}\nMedium error is {medium_error:.2f}')

In [ ]:
up_down_test = []
up_down_predicted = []

for y,x in zip(Y_test,X_test):
    if y[0] > x[-1][0]: up_down_test.append(1)
    elif y[0] < x[-1][0]: up_down_test.append(-1)
    else: up_down_test.append(0)
        
for y,x in zip(y_predicted,X_test):
    if y[0] > x[-1][0]: up_down_predicted.append(1)
    elif y[0] < x[-1][0]: up_down_predicted.append(-1)
    else: up_down_predicted.append(0)

In [ ]:
print(f'Trend accuracy is: {(sum(1 for r,p in zip(up_down_test,up_down_predicted) if r == p)/len(up_down_test))*100:.2f}%')

In [ ]:
print(f'Accuracy for upward trend is: {(sum(1 for r,p in zip(up_down_test,up_down_predicted) if r == 1 and p == 1)/sum(1 for v in up_down_test if v == 1))*100:.2f}%')
print(f'Accuracy for downward trend is: {(sum(1 for r,p in zip(up_down_test,up_down_predicted) if r == -1 and p == -1)/sum(1 for v in up_down_test if v == -1))*100:.2f}%')

##### Predictions over the last 92 days in test

In [ ]:
from matplotlib.pyplot import figure
figure(num=None, figsize=(40, 30), dpi=80, facecolor='w', edgecolor='k')

plt.plot(range(92), true_y_test[-92:])
plt.plot(range(92), predicted_y_test[-92:])
plt.show()

##### Last 8 days + prediction of last 4 days

In [ ]:
figure(num=None, figsize=(40, 30), dpi=80, facecolor='w', edgecolor='k')

plt.plot(range(8), true_y_test[-8:])
plt.plot(range(8), np.append(true_y_test[-8:-4], predicted_y_test[-4:]))
plt.show()